In [13]:
library(tidyverse)
library(repr)
library(tidymodels)

In [19]:
url <- "https://raw.githubusercontent.com/cpan0/project_proposal/main/diamonds.csv"
diamonds <- read_csv(url)
diamonds

Parsed with column specification:
cols(
  carat = col_double(),
  cut = col_character(),
  color = col_character(),
  clarity = col_character(),
  depth = col_double(),
  table = col_double(),
  x = col_double(),
  y = col_double(),
  z = col_double(),
  price = col_double()
)



carat,cut,color,clarity,depth,table,x,y,z,price
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.23,Ideal,E,SI2,61.5,55,3.95,3.98,2.43,326
0.21,Premium,E,SI1,59.8,61,3.89,3.84,2.31,326
0.23,Good,E,VS1,56.9,65,4.05,4.07,2.31,327
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.70,Very Good,D,SI1,62.8,60,5.66,5.68,3.56,2757
0.86,Premium,H,SI2,61.0,58,6.15,6.12,3.74,2757
0.75,Ideal,D,SI2,62.2,55,5.83,5.87,3.64,2757
